In [1]:
import pandas as pd
import numpy as np
import re
from scipy.stats import entropy
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import StandardScaler

### Cell 1 — Load Raw Dataset

This cell loads the original Champions Group dataset into memory.  
We also create a full raw copy (`df_raw_copy`) to ensure:
- Full reproducibility  
- Auditability of all later cleaning steps  
- Ability to compare "before vs after" during cleaning  


In [2]:
DATA_PATH = "../data/champions_group_data.csv"

# Load dataset
df_raw = pd.read_csv(DATA_PATH)

# Keep an untouched copy for auditing
df_raw_copy = df_raw.copy()

df_raw.shape

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_41816\356371746.py:4: DtypeWarning: Columns (26,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(DATA_PATH)


(8559, 72)

### Cell 2 — Define abbreviation detection logic

This cell defines the `is_abbreviation()` helper function used later to preserve uppercase abbreviations during cleaning.


In [3]:
def is_abbreviation(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.strip()
    if not t:
        return False

    # Short all-uppercase
    if t.isupper() and len(t) <= 12:
        return True

    # High uppercase ratio
    uppercase_ratio = sum(c.isupper() for c in t) / max(1, len(t))
    if uppercase_ratio > 0.6 and len(t) <= 18:
        return True

    # Pattern matching
    if re.match(r"^[A-Z0-9][A-Z0-9\-\./]*$", t) and len(t) <= 20:
        return True

    return False

### Cell 3 — Main Data Cleaning Function

This cell defines the core cleaning function used for preprocessing the dataset.  
It handles whitespace cleanup, pseudo-missing value replacement, safe numeric conversion,  
and controlled case normalization with abbreviation protection.


In [4]:
# Standardize pseudo-missing values
PSEUDO_MISSING = {
    "": pd.NA,
    "na": pd.NA,
    "n/a": pd.NA,
    "none": pd.NA,
    "null": pd.NA,
    "-": pd.NA
}

# Geographic fields → Title Case
GEO_COLUMNS = {
    "country",
    "country name",
    "country/region",
    "region",
    "state",
    "state/province",
    "parent state",
    "parent state/province",
    "global ultimate state",
    "global ultimate state/province",
    "domestic ultimate state",
    "domestic ultimate state/province",
}

# Business category fields → Title Case
BUSINESS_CATEGORY_COLUMNS = {
    "entity type",
    "ownership type",
    "company status",
    "company status (active/inactive)",
    "manufacturing status",
    "parent company status",
    "global ultimate company status",
    "domestic ultimate company status"
}

def clean_preserve_abbrev(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Identify original object columns
    obj_cols = df.select_dtypes(include="object").columns.tolist()

    # Whitespace + pseudo-missing standardization
    for col in obj_cols:
        s = df[col].astype("string")
        s = s.str.strip()
        s = s.str.replace(r"\s+", " ", regex=True)
        s = s.replace(PSEUDO_MISSING, regex=False)
        df[col] = s

    # Convert numeric-like columns to numeric
    for col in obj_cols:
        sample = df[col].dropna().astype(str).head(50)
        numeric_ratio = sample.str.match(r"^-?\d+(\.\d+)?$").mean()
        if numeric_ratio > 0.85:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Case normalization
    case_cols = df.select_dtypes(include=["object", "string"]).columns.tolist()

    for col in case_cols:
        s = df[col]
        col_lc = col.lower().strip()

        # Geographic fields → Title Case
        if col_lc in GEO_COLUMNS:
            df[col] = s.apply(lambda v: v.title() if isinstance(v, str) else v)
            continue

        # Business category fields → Title Case
        if col_lc in BUSINESS_CATEGORY_COLUMNS:
            df[col] = s.apply(lambda v: v.title() if isinstance(v, str) else v)
            continue

        # Franchise Status → leave as-is (no forced yes/no mapping)
        if col_lc == "franchise status":
            df[col] = s
            continue

        # General short-categorical cleanup (lowercase unless abbreviation)
        non_null = s.dropna()
        str_values = non_null[non_null.apply(lambda x: isinstance(x, str))]
        if str_values.empty:
            continue

        avg_len = str_values.map(len).mean()
        max_words = (str_values.str.count(" ").max() or 0) + 1

        if avg_len <= 30 and max_words <= 3:
            df[col] = s.apply(
                lambda v: (
                    v if not isinstance(v, str)
                    else (v if is_abbreviation(v) else v.lower())
                )
            )

    return df


# Clean dataset from the raw backup
df_clean = clean_preserve_abbrev(df_raw_copy)

### Cell 4 — Cleaning Audit

This cell compares each column before and after cleaning to measure how many
values were changed. It helps verify that the cleaning process is controlled,
minimal, and fully explainable.


In [5]:
audit_rows = []
for col in df_raw_copy.columns:

    # Only compare object-type columns
    if df_raw_copy[col].dtype == "object":

        # Convert both before/after to string for fair comparison
        before = df_raw_copy[col].astype("string")
        after  = df_clean[col].astype("string")

        # Count how many values changed
        changed = ((before.fillna("<<NA>>") != after.fillna("<<NA>>"))).sum()

         # Save results: column name, count of changes, ratio of changes
        audit_rows.append([col, int(changed), float(changed / len(df_raw_copy))])

# Convert to DataFrame and sort by highest change ratio
audit = pd.DataFrame(audit_rows, columns=["column", "cells_changed_count", "cells_changed_ratio"]) \
          .sort_values("cells_changed_ratio", ascending=False)

audit.head(15)


,column,cells_changed_count,cells_changed_ratio
27,Global Ultimate Country Name,8559,1.000000
20,Parent Country/Region,8559,1.000000
6,Country,8556,0.999649
10,SIC Description,8555,0.999533
15,Parent Company,8523,0.995794
28,Domestic Ultimate Company,8518,0.995210
21,Global Ultimate Company,8518,0.995210
18,Parent State/Province,8418,0.983526
0,Company Sites,8294,0.969038
31,Domestic Ultimate State/Province Name,8206,0.958757


### Cell 5 — Information Density Score (IDS)

This cell computes the Information Density Score for each column.  
IDS combines value diversity (entropy) and completeness (non-missing ratio)  
to measure how informative each column is for downstream analysis.


In [6]:
# Compute Information Density Score (IDS) for a column
def information_density(col: pd.Series) -> float:

    non_null = col.dropna()

    if non_null.empty or non_null.nunique() <= 1:
        return 0.0
    
    # Probability distribution of values
    probs = non_null.value_counts(normalize=True)

    # Entropy of distribution
    H = entropy(probs)

    # Normalized entropy
    H_norm = H / np.log(len(probs)) if len(probs) > 1 else 0.0
    return float((1 - col.isna().mean()) * H_norm)

# Build summary table for all columns
summary = pd.DataFrame({
    "dtype": df_clean.dtypes.astype(str),
    "missing_ratio": df_clean.isna().mean(),
    "n_unique": df_clean.nunique(dropna=True),
})
summary["IDS"] = df_clean.apply(information_density)

summary.sort_values("IDS", ascending=False).head(10)


,dtype,missing_ratio,n_unique,IDS
DUNS Number,int64,0.000000,8559,1.000000
Company Sites,string,0.000000,8557,0.999990
Parent Company,string,0.000000,7476,0.984701
Domestic Ultimate Company,string,0.000000,7203,0.979771
Global Ultimate Company,string,0.000000,7201,0.979698
Parent Street Address,string,0.012735,7339,0.971538
Address Line 1,string,0.040075,8156,0.959540
Entity Type,object,0.000000,3,0.952594
Domestic Ultimate Street Address,string,0.038790,6986,0.943380
Global Ultimate Street Address,string,0.039958,6967,0.941937


### Cell 6 — Missingness Structure Analysis (Mutual Information)

This cell evaluates whether missing values carry information by measuring
the association between missingness patterns and numeric variables using
Mutual Information (MI).


In [7]:
numeric_cols = df_clean.select_dtypes(include="number").columns.tolist()
missing_structure = {}

for col in df_clean.columns:

    # Missing ratio of the current column
    mr = df_clean[col].isna().mean()

    # Skip columns with low missingness or when no numeric columns exist
    if mr < 0.10 or len(numeric_cols) == 0:
        missing_structure[col] = 0.0
        continue

    # Binary indicator of missingness (1 = missing, 0 = observed)
    miss_flag = df_clean[col].isna().astype(int)
    mi_scores = []

    for nc in numeric_cols:

        # Only consider numeric columns with sufficient non-missing values
        valid = df_clean[nc].notna()
        if valid.sum() <= 50:
            continue
        try:

            # Discretize numeric values into quantile bins
            binned = pd.qcut(df_clean.loc[valid, nc], q=5, duplicates="drop")

             # Compute mutual information between missingness and numeric bins
            mi_scores.append(mutual_info_score(miss_flag.loc[valid], binned))
        except:
            continue
    
    # Use the maximum MI score as the missingness structure indicator
    missing_structure[col] = float(max(mi_scores)) if mi_scores else 0.0

summary["missing_structure_MI"] = pd.Series(missing_structure)
summary.sort_values("missing_structure_MI", ascending=False).head(10)


,dtype,missing_ratio,n_unique,IDS,missing_structure_MI
No. of Routers,object,0.385442,4,0.105415,0.364904
Phone Number,float64,0.804416,1670,0.195559,0.303823
Ownership Type,object,0.126417,5,0.011801,0.248285
No. of Servers,object,0.391284,3,0.005972,0.221281
No. of Storage Devices,object,0.401098,4,0.048510,0.216307
ANZSIC Code,float64,0.833392,136,0.130571,0.201196
ANZSIC Description,string,0.833158,137,0.130736,0.200499
Is Headquarters,float64,0.828601,2,0.097540,0.188650
NACE Rev 2 Description,string,0.823344,177,0.138340,0.179933
ISIC Rev 4 Description,string,0.823110,147,0.137654,0.179612


### Cell 7 — Redundancy Check (Correlation-Based)

This cell identifies redundant numeric features by computing the absolute
correlation matrix and flagging columns with correlation above 0.95.


In [8]:
# Set to store redundant (highly correlated) numeric columns
redundant = set()

if len(numeric_cols) >= 2:

    # Absolute correlation matrix for numeric features
    corr = df_clean[numeric_cols].corr().abs()

    # Identify redundant columns (correlation > 0.95)
    for i in range(len(corr.columns)):
        for j in range(i):
            if corr.iloc[i, j] > 0.95:
                redundant.add(corr.columns[i])
                
# Flag redundant columns in the summary table
summary["redundant"] = summary.index.isin(redundant)
summary[summary["redundant"]].head(20)


,dtype,missing_ratio,n_unique,IDS,missing_structure_MI,redundant
Employees Total,int64,0.000000,247,0.553791,0.000000,True
8-Digit SIC Code,float64,0.620283,445,0.299423,0.065698,True
Ticker,float64,0.999533,4,0.000467,0.002335,True
Longitude,float64,0.776609,326,0.179177,0.146916,True
Parent Postal Code,float64,0.022783,1583,0.779617,0.000000,True
Global Ultimate Postal Code,Int64,0.056665,1563,0.758436,0.000000,True
Domestic Ultimate Postal Code,float64,0.046618,1581,0.766902,0.000000,True
Registration Number,Float64,0.301087,5976,0.698882,0.135618,True
Is Domestic Ultimate,float64,0.823110,2,0.094748,0.179612,True
IT spend,int64,0.000000,1935,0.589199,0.000000,True


### Cell 8 — Final Column Decision Table

This cell generates a final KEEP / DROP / CONDITIONAL decision for each column
based on information density, missingness structure, redundancy, and uniqueness.


In [9]:
# Decision logic for whether to keep or drop each column
def decide(row):
    if row["n_unique"] <= 1:
        return "DROP: constant"

    # Drop columns with extremely high missingness and no information
    if (row["missing_ratio"] > 0.85) and (row["IDS"] < 0.05) and (row["missing_structure_MI"] < 0.01):
        return "DROP: no information"

    # Drop redundant numeric columns
    if bool(row["redundant"]):
        return "DROP: redundant"

    # Keep if missingness is informative (non-random)
    if row["missing_structure_MI"] > 0.02:
        return "KEEP: informative missingness"

    # Keep if information density is high
    if row["IDS"] > 0.15:
        return "KEEP: informative content"

    return "CONDITIONAL: review"

decision_table = summary.reset_index().rename(columns={"index": "column"}).copy()
decision_table["decision"] = summary.apply(decide, axis=1).values
decision_table["decision"].value_counts()


decision
KEEP: informative content        33
KEEP: informative missingness    19
DROP: redundant                   9
CONDITIONAL: review               5
DROP: constant                    5
DROP: no information              1
Name: count, dtype: int64

### Cell 10 — Export Cleaned Data, Audit Report, and Decision Table

This cell provides optional export commands for saving the cleaned dataset,
the cleaning audit log, and the final decision table.  
The lines are commented out so the user can enable them when needed

In [ ]:
# # File paths for optional exports
# CLEAN_OUT = "cleaned_base.csv"
# AUDIT_OUT = "cleaning_audit_changes.xlsx"
# DECISION_OUT = "ids_decision_table_no_dictionary.xlsx"

# # Exporting
# df_clean.to_csv(CLEAN_OUT, index=False)
# audit.to_excel(AUDIT_OUT, index=False)
# decision_table.to_excel(DECISION_OUT, index=False)